In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output  # pip install dash (version 2.0.0 or higher)
import mercantile
from shapely.wkt import loads
import json
import warnings


# ------------------------------------------------------------------------------
# Functions
lat_, long_= -12.074192, -77.042661
#lat_, long_= 4.710992, -74.072096

def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

def shapefile_to_geojson(data, index_list, level = 1, tolerance=0.0001):    
	geo_names = list(data['index'])
	geojson = {'type': 'FeatureCollection', 'features': []}
	for index in index_list:
		geo = data['geometry'][index].simplify(tolerance)
	
		if isinstance(geo.boundary, LineString):
			gtype = 'Polygon'
			bcoords = np.dstack(geo.boundary.coords.xy).tolist()
	
		elif isinstance(geo.boundary, MultiLineString):
			gtype = 'MultiPolygon'
			bcoords = []
			for b in geo.boundary:
				x, y = b.coords.xy
				coords = np.dstack((x,y)).tolist() 
				bcoords.append(coords) 
		else: pass

		feature = {'type': 'Feature',
                   'id' : index,
                   'properties': {'name': geo_names[index]},
                   'geometry': {'type': gtype,
                                'coordinates': bcoords},
					}
								
		geojson['features'].append(feature)
	return geojson

def plot_points(data):
	fig = px.scatter_mapbox(data, lat="latitude", lon="longitude", 
                        color='download_kbps',
                        #hover_name="City", 
                        #hover_data=["device_id"],
                        #color_discrete_sequence=["fuchsia"], 
                        #range_color = [-120,-80],
                        zoom=3, height=600,
                        )
	fig.update_layout(mapbox_style="light",mapbox_accesstoken=token)
	fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
	fig.show()
 

def plot_bins(cell_gdf_,col, zmin,zmax):
	a = list(cell_gdf_.index)
	data_geojson = shapefile_to_geojson(cell_gdf_,a)
	df_geo = pd.DataFrame(cell_gdf_)

	fig = go.Figure(go.Choroplethmapbox(geojson=data_geojson, 
                    locations=df_geo.index, 
                    z=df_geo[col],
                    colorscale = 'Cividis',
                    #colorscale=['Blue','Red'],
                    #color_continuous_scale=["red", "green", "blue"],
                    marker_line_width=0,
                    zmin=zmin,
                    zmax=zmax,
                    text=df_geo['tile_id'],
                    
                    reversescale=True,
                    marker_opacity=0.7,
                    showscale=True))
	fig.update_layout(mapbox_style="light", 
					mapbox_accesstoken=token,
					mapbox_zoom=12  , 
                    height=700,
					mapbox_center = {"lat": lat_, "lon": long_},
					margin={"r":0,"t":0,"l":0,"b":0},
					legend_font_color ='white',
					coloraxis=dict(
							colorbar=dict(
									tickcolor='white',
									title='Titulo'
									)
							),
					
					)

	#fig.update_coloraxes(colorbar_tickfont_color='white')
	#fig.update_traces(colorbar_tickfont_color='black', selector=dict(type='choroplethmapbox'))
	fig.update_traces(colorbar_tickfont_color='black',
												colorbar_title_text=col,
												colorbar_title_font_color = 'black',                           
												selector=dict(type='choroplethmapbox'),
												colorbar_len=0.4)

	fig.show()


def check_consistency(row):
  if (row['download_kbps']>=25000) & (row['upload_kbps']>=3000):
    return 1
  else:
    return 0

def user_ap_value(row):
  if (row['AP_value']/row['result_id']>=0.5):
    return 1
  else:
    return 0

def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None

colorscales_dl_th = [
    ((0.0, '#FF0000'), (1.0, '#FF0000')),
    ((0.0, '#FFA500'), (1.0, '#FFA500')),
    ((0.0, '#dbdee9'), (1.0, '#dbdee9')),
    ((0.0, '#949ebe'), (1.0, '#949ebe')),
    ((0.0, '#4e6294'), (1.0, '#4e6294')),
    
]

colors_rsrp = [

    ((0.0, '#FF0000'), (1.0, '#FF0000')),
    ((0.0, '#FFA500'), (1.0, '#FFA500')),
    ((0.0, '#ffff00'), (1.0, '#ffff00')),
    ((0.0, '#D7BDE2'), (1.0, '#D7BDE2')),
    ((0.0, '#1065c0'), (1.0, '#1065c0'))


]

colorscales_test_count = [
    
    ((0.0, '#77cbd2'), (1.0, '#77cbd2')),
    ((0.0, '#5a97ab'), (1.0, '#5a97ab')),
    ((0.0, '#323fca'), (1.0, '#323fca')),
    ((0.0, '#151d35'), (1.0, '#151d35')),
    ((0.0, '#878131'), (1.0, '#878131'))
]

colorscales_web = [
    ((0.0, '#269142'), (1.0, '#269142')),
    ((0.0, '#35cc5d'), (1.0, '#35cc5d')),
    #((0.0, '#717fa9'), (1.0, '#717fa9')),
    ((0.0, '#9ae85f'), (1.0, '#9ae85f')),
    ((0.0, '#e33030'), (1.0, '#e33030'))]



def plot_categorical(bis_freq_att,col_cat,col,flag_,map_):
    if flag_ == 'radio':
        colorscales_ = colors_rsrp
    elif flag_ == 'test':
        colorscales_ = colorscales_test_count
    elif flag_ == 'web':
        colorscales_ = colorscales_web
    else:
        colorscales_ = colorscales_dl_th
    
    fig = go.Figure()
    for i, winner in enumerate(bis_freq_att.sort_values(col)[col_cat].unique()):
        dfp = bis_freq_att.loc[bis_freq_att[col_cat] == winner]
        dfp.reset_index(inplace=True)
        dfp.drop('index',axis=1,inplace=True)
        dfp.reset_index(inplace=True)
        a1 = list(dfp.index)
        data_geojson1 = shapefile_to_geojson(dfp,a1)
        df_geo1 = pd.DataFrame(dfp)


        fig.add_choroplethmapbox(geojson=data_geojson1, locations=df_geo1['index'],
                                z=[i,] * len(dfp), 
                                showlegend=True, name=winner,
                                marker_line_width=0,
                                marker_opacity = 0.5,
                                #text = df_geo1['tile_id'],
                                colorscale=colorscales_[i], showscale=False)

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                    mapbox_style=map_,
                    mapbox_accesstoken=token,
                    mapbox_zoom=10.5,
                    height=850,
                    #, 
                    mapbox_center = {"lat": lat_, "lon":long_},)

    fig.update_layout(legend=dict(
        yanchor="bottom",
        x=0.99,
        xanchor="right",
        y=0.8
    ))

    return fig
    
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import hdbscan

def clustering_dbscan(type,data_hotspot,eps_,min_samples_,min_cluster_size_,min_samples__):

    data_hotspot["x"] = data_hotspot.centroid.x
    data_hotspot["y"] = data_hotspot.centroid.y

    #Algoritmo DBSCAN para agrupar puntos densos
    #grilla_arieso = cell_gdf_.copy()
    X = data_hotspot[['y','x']].values
    # Normalizamos valores y aplicamos DBSCAN
    Clus_dataSet = StandardScaler().fit_transform(X)
    from sklearn.cluster import DBSCAN
    #clustering = hdbscan.HDBSCAN(min_cluster_size=min_samples_,gen_min_span_tree=True).fit(Clus_dataSet)
    if type=='N':
        clustering = DBSCAN(eps=eps_, min_samples=min_samples_).fit(Clus_dataSet)
    else:
        clustering = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size_,min_samples=min_samples__).fit(Clus_dataSet)

    data_hotspot['cluster']=clustering.labels_
    data_hotspot['cluster'] =data_hotspot['cluster'].astype('str')
    data_hotspot.loc[data_hotspot['cluster']=='-1','cluster']='200'

    fig = px.scatter_mapbox(data_hotspot, 
                            lat="y", 
                            lon="x",                        
                            color= 'cluster', 
                            zoom=11.3, 
                            color_discrete_sequence=px.colors.qualitative.Alphabet,
                            height=780       
                                    
                            #40.740779, -73.853616   
                        )
    fig.update_layout(mapbox_style="light",mapbox_accesstoken=token, margin={"r":0,"t":0,"l":0,"b":0},mapbox_center = {"lat": lat_, "lon":long_} )
    fig.show()

def view_polygons(data_pols):
    dframe_pol= data_pols.copy()
    dframe_pol['color'] = 10
    dframe_pol.reset_index(inplace=True)
    df_pol = pd.DataFrame(dframe_pol)
    a = list(df_pol.index)
    data_geojson_pol = shapefile_to_geojson(dframe_pol,a)
    
    poligonos_dra = go.Choroplethmapbox(geojson=data_geojson_pol, 
                                        locations=df_pol.index, 
                                        z=df_pol['color'],
                                        #colorscale="Viridis",
                                        marker_line_width=0,
                                        #colorscale=['yellow','green'],
                                        showscale=False,                                    
                                        #opacity=0.5,
                                        #reversescale=True,
                                        marker_opacity=0.7
                                        )
    
    layout = go.Layout(mapbox_style="light", 
                            mapbox_accesstoken=token,
                            mapbox_zoom=11.3, 
                        
                            mapbox_center = {"lat": lat_, "lon":long_},
                            height=800,
                            showlegend=True,
                            
                            margin={"r":0,"t":0,"l":0,"b":0}
                            ) 

    fig_mapa = go.Figure(data=[poligonos_dra], layout =layout)
    fig_mapa.update_traces(colorbar_tickfont_color='black',
                            #colorbar_title_text=columna,
                            colorbar_title_font_color = 'black',                           
                            selector=dict(type='choroplethmapbox'),
                            colorbar_len=0.4)

   # fig_mapa.show()
    return fig_mapa

colorscales_cat_ranking = [
    ((0.0, '#98f9f0'), (1.0, '#98f9f0')),
    ((0.0, '#77cbd2'), (1.0, '#77cbd2')),
    ((0.0, '#5a97ab'), (1.0, '#5a97ab')),
    ((0.0, '#323fca'), (1.0, '#323fca')),
    ((0.0, '#151d35'), (1.0, '#151d35')),
    ((0.0, '#151d35'), (1.0, '#151d35'))
]



colorscales_cat_ranking = [
    ((0.0, '#98f9f0'), (1.0, '#98f9f0')),
    ((0.0, '#77cbd2'), (1.0, '#77cbd2')),
    ((0.0, '#5a97ab'), (1.0, '#5a97ab')),
    ((0.0, '#323fca'), (1.0, '#323fca')),
    ((0.0, '#151d35'), (1.0, '#151d35')),
    ((0.0, '#151d35'), (1.0, '#151d35'))
]


def plot_categorical_ranking(bis_freq_att,col_cat,col,maptype_):
    fig = go.Figure()
    for i, winner in enumerate(bis_freq_att.sort_values(col)[col_cat].unique()):
        dfp = bis_freq_att.loc[bis_freq_att[col_cat] == winner]
        dfp.reset_index(inplace=True)
        dfp.drop('index',axis=1,inplace=True)
        dfp.reset_index(inplace=True)
        a1 = list(dfp.index)
        data_geojson1 = shapefile_to_geojson(dfp,a1)
        df_geo1 = pd.DataFrame(dfp)


        fig.add_choroplethmapbox(geojson=data_geojson1, locations=df_geo1['index'],
                                z=[i,] * len(dfp), 
                                showlegend=True, name=winner,
                                marker_line_width=0,
                                colorscale=colorscales_cat_ranking[i], showscale=False)

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                    #39.104584, -98.303019
                    mapbox_style=maptype_, 
                    mapbox_accesstoken=token,
                    mapbox_zoom=4, 
                    height=800,
                    #, 
                    mapbox_center = {"lat": lat_, "lon":long_},)

    fig.update_layout(legend=dict(
        yanchor="bottom",
        x=0.99,
        xanchor="right",
        y=0.1
    ))

    fig.show()
    
colorscales_cat_index = [
    
    ((0.0, '#e4e4e5'), (1.0, '#e4e4e5')),
    ((0.0, '#a3c3df'), (1.0, '#a3c3df')),
    ((0.0, '#907bc7'), (1.0, '#907bc7')),
    ((0.0, '#6b6384'), (1.0, '#6b6384')),
    ((0.0, '#4c1130'), (1.0, '#4c1130'))
]

def plot_categorical_index(bis_freq_att,col_cat,col,map_):
    fig = go.Figure()
    for i, winner in enumerate(bis_freq_att.sort_values(col)[col_cat].unique()):
        dfp = bis_freq_att.loc[bis_freq_att[col_cat] == winner]
        dfp.reset_index(inplace=True)
        dfp.drop('index',axis=1,inplace=True)
        dfp.reset_index(inplace=True)
        a1 = list(dfp.index)
        data_geojson1 = shapefile_to_geojson(dfp,a1)
        df_geo1 = pd.DataFrame(dfp)


        fig.add_choroplethmapbox(geojson=data_geojson1, locations=df_geo1['index'],
                                z=[i,] * len(dfp), 
                                showlegend=True, name=winner,
                                marker_line_width=0.5,
                                #text=df_geo1['sector'],
                                marker_opacity=0.8,
                                colorscale=colorscales_cat_index[i], showscale=False)

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                    #39.104584, -98.303019
                    mapbox_style=map_, 
                    mapbox_accesstoken=token,
                    mapbox_zoom=12, 
                    height=850,
                    #, 
                    mapbox_center = {"lat": lat_, "lon":long_},)

    fig.update_layout(legend=dict(
        yanchor="bottom",
        x=0.99,
        xanchor="right",
        y=0.8
    ))

    return fig

def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()

#Creating tiles from lat long


import math
def deg2num(lat_deg, lon_deg, zoom):
  lat_rad = math.radians(lat_deg)
  n = 2.0 ** zoom
  xtile = int((lon_deg + 180.0) / 360.0 * n)
  ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
  return (xtile, ytile)

def f_quadkey(row, zoom):
    lat_deg = row['latitude']
    lon_deg = row['longitude']
    x,y = deg2num(lat_deg,lon_deg,zoom)

    key = ''
    for i in range(zoom, 0, -1):
        val = 0
        mask = 1 << (i - 1)

        if((x & mask) != 0):
            # move to second column
            val += 1
        if((y & mask) != 0):
            # move to second row
            val += 2
        key += str(val)
    return key

def f_tile_x2lon_(x,zoom):
    cad = x/ (1 << zoom) * 360.0 - 180.0
    #return np.round(cad,5)
    return cad

def f_tile_y2lat_(y,zoom):
    cad = np.arctan(0.5 * np.exp(np.pi * (1 - 2 * y / (1 << zoom))) - 0.5 * np.exp(-(np.pi * (1 - 2 * y/ (1 << zoom)))))
    #return np.round(np.degrees(cad),5)
    return np.degrees(cad)

def f_tile_xyz2wkt_(row, zoom):
    lat_deg = row['latitude']
    lon_deg = row['longitude']

    x,y = deg2num(lat_deg,lon_deg,zoom)
    cad = 'POLYGON((' + str(f_tile_x2lon_(x,zoom)) +' ' + str(f_tile_y2lat_(y,zoom)) + ', ' + str(f_tile_x2lon_(x+1,zoom)) + ' ' + str(f_tile_y2lat_(y,zoom)) + ', ' + str(f_tile_x2lon_(x+1,zoom)) + ' ' + str(f_tile_y2lat_(y+1,zoom)) + ', ' + str(f_tile_x2lon_(x,zoom)) + ' ' + str(f_tile_y2lat_(y+1,zoom)) + ', ' + str(f_tile_x2lon_(x,zoom)) + ' ' + str(f_tile_y2lat_(y,zoom)) + '))'
    return cad

def f_quadkey_to_x_(quad):
    x=0
    z=len(quad)
    for i in range(z, 0, -1):
        mask = 1 << (i - 1)
        q = int(quad[z-i])

        if(q==1 or q==3):
            x = x | mask

    return x

def f_quadkey_to_y_(quad):
    y = 0
    z = len(quad)
    for i in range(z, 0, -1):
        mask = 1 << (i - 1)
        q = int(quad[z-i])

        if(q==2 or q==3):
            y = y | mask

    return y


def f_quadket2wkt_(quad):
    x=f_quadkey_to_x_(quad)
    y=f_quadkey_to_y_(quad)
    zoom=len(quad)
    cad = 'POLYGON((' + str(f_tile_x2lon_(x,zoom)) +' ' + str(f_tile_y2lat_(y,zoom)) + ', ' + str(f_tile_x2lon_(x+1,zoom)) + ' ' + str(f_tile_y2lat_(y,zoom)) + ', ' + str(f_tile_x2lon_(x+1,zoom)) + ' ' + str(f_tile_y2lat_(y+1,zoom)) + ', ' + str(f_tile_x2lon_(x,zoom)) + ' ' + str(f_tile_y2lat_(y+1,zoom)) + ', ' + str(f_tile_x2lon_(x,zoom)) + ' ' + str(f_tile_y2lat_(y,zoom)) + '))'
    return cad



# ------------------------------------------------------------------------------


app = Dash(__name__)

server = app.server

# ------------------------------------------------------------------------------
# -- Import and clean data (importing csv into pandas)

# ruta_archivos = "Input PYTHON/*.csv"

# # Lista de archivos
# archivos = glob.glob(ruta_archivos)

# # Leer todos los archivos y combinarlos en un solo DataFrame
# df = pd.concat([pd.read_csv(archivo) for archivo in archivos], ignore_index=True)

df = pd.read_csv("All tech Guatemala coverage - Dec2024.csv")

df = df[df['mno_name'].isin(['Tigo', 'Claro'])]

# ------------------------------------------------------------------------------
#create geometry
df['quadkey'] = df.apply(lambda row: f_quadkey(row, 17), axis=1)
df['geometry'] = df['quadkey'].apply(f_quadket2wkt_)

print(df[:5])

# ------------------------------------------------------------------------------
#create pivot table
# Crear la tabla pivote
pivot_table = pd.pivot_table(
    data=df,  # DataFrame de origen
    index=['technology', 'quadkey', 'geometry'],  # Índices
    columns=['mno_name'],  # Columnas
    values=['scan_count', 'signal_level'],  # Valores
    aggfunc={'scan_count': 'sum', 'signal_level': 'median'}  # Funciones de agregación
)


pivot_table.columns = [f'{col[0]}_{col[1]}' for col in pivot_table.columns]
pivot_table.reset_index(inplace=True)
pivot_table


nuevos_nombres = [  'technology','quadkey', 'geometry',
                    'scans_claro','scans_tigo','signal_claro','signal_tigo']

# Asignar los nuevos nombres de columnas al DataFrame
pivot_table.columns = nuevos_nombres


# ------------------------------------------------------------------------------
#technology coverage comparison

def condicion_if(row):
    if pd.notna(row['scans_claro']) and pd.notna(row['scans_tigo']):
        return 'Ambos Tigo + Claro'
    elif pd.isnull(row['scans_claro']) and pd.notna(row['scans_tigo']):
        return 'Solo Tigo'
    elif pd.notna(row['scans_claro']) and pd.isnull(row['scans_tigo']):
        return 'Solo Claro'
    else:
        return 'NA'

pivot_table['comparison'] = pivot_table.apply(condicion_if, axis=1)


# ------------------------------------------------------------------------------

# Tu función para convertir Quadkeys en geometrías
def quadkey_to_geometry(quadkey):
    tile = mercantile.quadkey_to_tile(quadkey)
    bbox = mercantile.bounds(tile)
    polygon_wkt = f"POLYGON(({bbox.west} {bbox.north}, {bbox.east} {bbox.north}, {bbox.east} {bbox.south}, {bbox.west} {bbox.south}, {bbox.west} {bbox.north}))"
    return loads(polygon_wkt)

# Aplica la función a la columna Quadkey para obtener geometrías
pivot_table['geometry'] = pivot_table['quadkey'].apply(quadkey_to_geometry)
pivot_table

#Creates geometry format for Pivot Table
gdf_csv = gpd.GeoDataFrame(pivot_table, geometry="geometry")

# Mostrar la tabla pivote ordenada

if gdf_csv.crs is None:
    gdf_csv.set_crs(epsg=4326, inplace=True)

#gdf_csv.plot(column='quadkey')

# ------------------------------------------------------------------------------
# App layout
app.layout = html.Div([

    html.H1("Comparación Cobertura Móvil Tigo y Claro - Guatemala. Periodo: Octubre 2024 a Marzo 2025", style={'text-align': 'center'}),

    dcc.Dropdown(
        id="slct_tech",
        options=[
            {"label": "5G", "value": 'NR'},
            {"label": "4G", "value": 'LTE'},
            {"label": "WCDMA", "value": 'WCDMA'},
            {"label": "CDMA", "value": 'CDMA'},
            {"label": "GSM", "value": 'GSM'}
        ],
        multi=False,
        value='LTE',
        style={'width': "40%"}
    ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_tech_map', figure={})

])



# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
warnings.filterwarnings("ignore", category=FutureWarning)

@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_tech_map', component_property='figure')],
#     Output(component_id='bar_plot', component_property='figure')],
    [Input(component_id='slct_tech', component_property='value')]
    )

def update_graph(option_slctd):
    print(option_slctd)

    container = f"The selected technology was: {option_slctd}"

    # Filtrar GeoDataFrame por tecnología
    dff = gdf_csv[gdf_csv["technology"] == option_slctd]

    if dff.empty:
        print("GeoDataFrame is empty.")
        return container, go.Figure()

    # Convertir GeoDataFrame a GeoJSON
    geojson = json.loads(dff.to_json())

    # Reproyectar temporalmente para calcular centroides correctos
    gdf_projected = dff.to_crs(epsg=3857)
    centroid = gdf_projected.geometry.centroid.to_crs(epsg=4326)

    # Crear figura
    fig = px.choropleth_mapbox(
        dff,
        geojson=geojson,
        locations=dff.index,
        color="comparison",  # Cambia a otra columna si prefieres
        mapbox_style="carto-positron",
        center={
            "lat": centroid.y.mean(),
            "lon": centroid.x.mean()
        },
        zoom=11,  # Ajusta según nivel de detalle de tus tiles
        opacity=0.6,
        hover_name="quadkey"
    )

    fig.update_layout(
        width=1700,
        height=900,
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
    )

    return container, fig

# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=True)

  mno_name technology            quadkey  \
1    Claro        LTE  02331312103330323   
2    Claro        LTE  02331312131210112   
3    Claro        LTE  02331313033213332   
4    Claro        LTE  02331312131210133   
5    Claro      WCDMA  02331313300111010   

                                            geometry   latitude  longitude  \
1  POLYGON((-91.78802490234375 14.968666650866508...  14.967372 -91.787038   
2  POLYGON((-91.5216064453125 14.857195658870621,...  14.855425 -91.520572   
3  POLYGON((-90.7965087890625 14.650025687128654,...  14.648266 -90.794954   
4  POLYGON((-91.51885986328125 14.851886077521906...  14.850875 -91.517641   
5  POLYGON((-90.5438232421875 14.604847155053898,...  14.604017 -90.542826   

   signal_level  user_count  scan_count  
1    -96.788727          73        2539  
2    -85.287582          82        2472  
3   -103.389588          43        3424  
4    -88.195839          77        2221  
5    -76.156250          15        1876  


LTE
LTE
LTE
